## Expose Parallelism

### Learning Objectives

This lab focuses on the single most important factor for achieving good performance on NVIDIA GPUs: expose massive parallelism. We will:

- Learn the architecture of Streaming Multiprocessors (SMs) in NVIDIA GPUs
- Study the latency hiding nature of GPUs
- Gain a deeper understanding of the CUDA execution model (how warps and blocks map to the GPU architecture)
- Understand how to select optimal kernel launch configurations

After completing this lab, you will be able to understand how to map the parallelism of your problem to the GPU architecture, so you can take full advantage of the GPU's processing power.

Note that in general you want to think about code performance in terms of an analysis-driven optimization process: the idea is to use profiling tools to determine what is the performance limiter for your code, and then rewrite code to remove that bottleneck. We will be discussing how to engage in this methodical process in a later module. The reason we're starting with a focus on optimization concepts now is that if you fully write your code and only then begin to profile it, there may be only so much performance you can unlock, due to the architecture of your code not matching well to the architecture of the GPU. In this lab we're going to describe the GPU architecture so that you can write your code from the start in a way that maps well to the GPU architecture.

### Prerequisites

It is assumed that participants understand the fundamental principles of working with CUDA code, including:

- How to launch CUDA kernels that use both blocks and threads


- Basic memory management (`cudaMalloc()`, `cudaFree()`, `cudaMemcpy`)


- How to compile and run CUDA code

## NVIDIA GPU Architecture

The primary concept we will focus on throughout this lab is the following: **in order to use GPUs effectively, you must expose massive parallelism** in your program. Specifically, **your program must launch a lot of threads**.

To understand why that is, let's take a look at the hardware architecture of NVIDIA GPUs. It is true for GPUs as it is true for other processors that you will need to write your code in a way that maps well to the GPU's architecture to achieve maximum performance.

<img src="images/A100.png" width="800" />

A NVIDIA GPU can largely be defined in terms of a streaming multiprocessor (SM). SMs are the building blocks for GPUs and define the characteristics, behavior, and instruction set of each GPU. When we build a large GPU of a given architecture, to a first approximation, we do that by dropping a large number of SMs on a die. When we build a small GPU of a given architecture, to a first approximation, we do that by dropping a small number of SMs on a die. Smaller GPUs with fewer SMs typically have lower performance. The block diagram for A100 below shows an architecture with 128 SMs (though only 108 are actually enabled in a production A100 chip).

<img src="images/A100_block_diagram.png" width="800" />

### Kepler SM (Compute Capability 3.5)

An SM of the Kepler generation has the following characteristics:

- 192 single precision (SP) units ("CUDA cores")
- 64 double precision (DP) units
- LD/ST units, 64K registers
- 4 warp schedulers
- K20: 13 SMs, 5 GB
- K20X: 14 SMs, 6 GB
- K40: 15 SMs, 12 GB

We think about GPU microarchitecture a little bit differently than CPUs. In marketing materials about NVIDIA GPUs you will often see the term "cores" or "CUDA cores" used. It is useful to note that this is not really the same concept as a traditional CPU core. In fact a CPU core might be best compared to an SM. A GPU core is most analogous to an arithmetic logic unit (ALU) or floating point unit (FPU) of a CPU core. These single precision units have a very small set of operations they can implement: essentially only floating point adds, multiplies, and fused multiply-adds (FMAs) on 32-bit floating point numbers. The double precision units do the same on 64-bit floating point data. Note that in general, there are a different number of SP and DP units on each SM.

<img src="images/kepler_sm.png" width="800" />

GPUs are for the most part defined by a load/store architecture. Almost all instructions involve first loading data from memory into registers, doing some computation in registers, and then storing the result to memory.

The warp schedulers are the instruction dispatch units for the SM: they decide when and which instructions get executed on a per-warp basis. (We haven't defined what a "warp" is yet, but we will soon.)

### Maxwell / Pascal SM (Compute Capability 5.2, 6.1)

Let's look now at one example from the Maxwell generation of NVIDIA GPUs. (It turns out that this particular SM was architecturally quite similar to those used in some variants of the Pascal generation, so we can group those here as well.)

- 128 SP units ("CUDA cores")
- 4 DP units
- LD/ST units
- INT8 instructions (in the Pascal variant)
- 4 warp schedulers
- M40: 24 SMs, 12/24 GB
- P40: 30 SMs, 24 GB
- P4: 20 SMs, 8 GB

<img src="images/maxwell_sm.png" width="800" />

Note that compared to the previous SM, the ratio of DP to SP units is much smaller. Note also that the number of SP units per SM is smaller than for the Kepler architecture.

### Pascal / Volta / Ampere SM (Compute Capability 6.0, 7.0, 8.0)

- 64 SP units ("CUDA cores")
- 32 DP units
- LD/ST units
- FP16 at twice the SP rate
- Pascal: 2 warp schedulers; Volta, Ampere: 4 warp schedulers
- Tensor Cores (in the Volta and Ampere variants)
- INT32 units (in the Volta and Ampere variants)
- P100: 56 SMs, 16 GB
- V100: 80 SMs, 16/32 GB
- A100: 108 SMs, 40 GB

<img src="images/volta_sm.png" width="800" />

Notice that the number of SMs has been growing substantially over time, so we are forced to write code that is flexible enough to run efficiently even on massively parallel architectures. For example, the NVIDIA A100 GPU has 108 SMs compared to V100's 80, and well written CUDA code developed for V100 should be able to scale effectively to A100 without requiring anything other than recompiling.

## Exercise: Grid and Block Size Experimentation

Let's empirically learn how performance depends on the number of CUDA threads and blocks we launch. We could do analysis like this using standard timing methods in host code, but we'll introduce a new concept, using a GPU profiler. In a future session, we'll more comprehensively learn about the GPU profilers (Nsight Compute and Nsight Systems), but for now we will use Nsight Compute in a fairly simple fashion to get some basic data about kernel behavior, to use for comparison. (If you'd like to read more about NVIDIA developer tools, check out [our overview](https://developer.nvidia.com/tools-overview).)

Our (fully implemented) code implements vector addition: [exercises/vector_add.cu](exercises/vector_add.cu). Relative to the typical first implementation of vector addition, the only twist is that our kernel uses a technique called a grid-stride loop. This is a flexible kernel design method that allows a simple kernel to handle an arbitrary size data set with an arbitrary size kernel launch configuration. If you'd like to read more about grid-stride loops, check out the [NVIDIA Developer Blog](https://devblogs.nvidia.com/cuda-pro-tip-write-flexible-kernels-grid-stride-loops/).

The code has these two lines in it that control the grid sizing:

```
  int blocks = 1;  // modify this line for experimentation
  int threads = 1; // modify this line for experimentation
```

The first variable `blocks` chooses the total number of blocks to launch. The second variable `threads` chooses the number of threads per block to launch. This second variable must be constrained to choices between 1 and 1024, inclusive. These are limits imposed by the GPU hardware.

Let's consider three cases. In each case, we will modify the `blocks` and `threads` variables, recompile the code, and then run the code under the Nsight Compute profiler (whose command line name is `ncu`).

### a) 1 block, 1 thread per block

For this experiment, leave the code as you have created it to complete exercise 1 above. When running the code you may have noticed it takes a few seconds to run. Suppose we want to measure exactly how long the kernel takes. The profiler can help us answer that question, and we can use this duration (or various other characteristics) as indicators of "performance" for comparison. The kernel is designed to do the same set of arithmetic calculations regardless of the grid sizing choices, so we can say that shorter kernel duration corresponds to higher performance.

If you'd like to get a basic idea of "typical" profiler output, you could use the following command:

```
ncu ./vector_add
```

However for this 1 block/1 thread test case, the profiler will spend several minutes assembling the requested set of information. Since our focus is on kernel duration, we can use a command that allows the profiler to run more quickly:

```
ncu  --section SpeedOfLight ./vector_add
```

This will allow the profiler to complete its work in under a minute by collecting only a small set of statistics.

We won't parse all the output, but we're interested in these lines:

```
Duration                                          second                           2.86
```

and:

```
Memory [%]                                             %                           0.04
...
SM [%]                                                 %                           0.04
```

The above indicate that our kernel took about 3 seconds to run and achieved much less than 1% of the peak throughput of the memory and compute (SM) subsystems of the GPU. Can we improve the situation with some changes to our grid sizing?

In [4]:
!nvcc -arch=native -o vector_add exercises/vector_add.cu; sudo ncu --section SpeedOfLight ./vector_add

==PROF== Connected to process 407902 (/home/sanjay42/sanjay/cuda/ATLAS_CUDA_tutorial/CUDA/02_Expose_Parallelism/vector_add)
Current device: 0
Device name: NVIDIA GeForce RTX 4060 Ti
Total global memory: 16453568 KB
Total constant memory: 64 KB
Total shared memory per block: 48 KB
==PROF== Profiling "vadd" - 0: 0%....50%....100% - 9 passes
A[0] = 0.840188
B[0] = 0.394383
C[0] = 1.234571
==PROF== Disconnected from process 407902
[407902] vector_add@127.0.0.1
  vadd(const float *, const float *, float *, int) (1, 1, 1)x(1, 1, 1), Context 1, Stream 7, Device 0, CC 8.9
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- --------------
    Metric Name               Metric Unit   Metric Value
    ----------------------- ------------- --------------
    DRAM Frequency          cycle/nsecond           8.74
    SM Frequency            cycle/nsecond           2.31
    Elapsed Cycles                  cycle  4,914,894,913
    Memory Throughput                   %   

### b) 1 block, 1024 threads per block

Our working hypothesis will be that we want to use lots of threads. More specifically we learned that we'd like to deposit as many as 2048 threads on a single SM, and ideally we'd like to do this across all the SMs in the GPU. This allows the GPU to better use the SM and memory resources of the GPU (we will soon understand more about why this is).

So let's take a baby step with our code. Let's change from 1 block and 1 thread to 1 block with 1024 threads. As we've learned, this structure isn't very good, because it can use at most a single SM on our GPU, but can it improve performance at all?

Edit the code to make the changes to the `threads` (= 1024) variable only. Leave the `blocks` variable at 1. Recompile the code and then rerun the same profiler command. What are the kernel duration and (achieved) memory throughput now?

(You should now observe a kernel duration that drops from the second range to the millisecond range, and the memory throughput should now be substantially higher.)

In [5]:
!nvcc -arch=native -o vector_add exercises/vector_add.cu; sudo ncu --section SpeedOfLight ./vector_add

==PROF== Connected to process 408375 (/home/sanjay42/sanjay/cuda/ATLAS_CUDA_tutorial/CUDA/02_Expose_Parallelism/vector_add)
Current device: 0
Device name: NVIDIA GeForce RTX 4060 Ti
Total global memory: 16453568 KB
Total constant memory: 64 KB
Total shared memory per block: 48 KB
==PROF== Profiling "vadd" - 0: 0%....50%....100% - 9 passes
A[0] = 0.840188
B[0] = 0.394383
C[0] = 1.234571
==PROF== Disconnected from process 408375
[408375] vector_add@127.0.0.1
  vadd(const float *, const float *, float *, int) (1, 1, 1)x(1024, 1, 1), Context 1, Stream 7, Device 0, CC 8.9
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- ------------
    Metric Name               Metric Unit Metric Value
    ----------------------- ------------- ------------
    DRAM Frequency          cycle/nsecond         8.74
    SM Frequency            cycle/nsecond         2.31
    Elapsed Cycles                  cycle   23,399,703
    Memory Throughput                   %        14.1

### c) 160 blocks, 1024 threads per block

Let's fill the GPU now. We learned that an NVIDIA V100 has 80 SMs, and each SM can handle at most 2048 threads. If we create a grid of 160 blocks, each of 1024 threads, this should allow for maximum utilization ("occupancy") of our kernel/grid on the GPU. Make the necessary changes to the `blocks` (= 160) variable (the `threads` variable should already be at 1024 from exercise b), recompile the code, and rerun the profiler command as given in exercise a). What is the performance (kernel duration) and achieved memory throughput now?

(You should now observe a kernel duration that has dropped to the microsecond range - ~500us - and a memory throughput that should be fairly close to the peak theoretical throughput of 100%.)

In [4]:
!nvcc -arch=native -o vector_add exercises/vector_add.cu; sudo ncu --section SpeedOfLight ./vector_add

==PROF== Connected to process 849826 (/home/sanjay42/sanjay/cuda/ATLAS_CUDA_tutorial/CUDA/02_Expose_Parallelism/vector_add)
Current device: 0
Device name: NVIDIA GeForce RTX 4060 Ti
Total global memory: 16453568 KB
Total constant memory: 64 KB
Total shared memory per block: 48 KB


==PROF== Profiling "vadd" - 0: 0%....50%....100% - 9 passes
A[0] = 0.840188
B[0] = 0.394383
C[0] = 1.234571
==PROF== Disconnected from process 849826
[849826] vector_add@127.0.0.1
  vadd(const float *, const float *, float *, int) (816, 1, 1)x(1024, 1, 1), Context 1, Stream 7, Device 0, CC 8.9
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- ------------
    Metric Name               Metric Unit Metric Value
    ----------------------- ------------- ------------
    DRAM Frequency          cycle/nsecond         8.69
    SM Frequency            cycle/nsecond         2.30
    Elapsed Cycles                  cycle    3,494,558
    Memory Throughput                   %        92.06
    DRAM Throughput                     %        92.06
    Duration                      msecond         1.52
    L1/TEX Cache Throughput             %        10.37
    L2 Cache Throughput                 %        18.05
    SM Active Cycles                cycle 3,463,740.44
  

### d) Vary threads per block

Keeping the total size of the grid fixed at 160 * 1024 = 163,840 threads, vary the number of threads per block; the constraint requires you to inversely vary the number of blocks. What is the best choice for this problem?

In [13]:
!nvcc -arch=native -o vector_add exercises/vector_add.cu; sudo ncu --section SpeedOfLight ./vector_add

==PROF== Connected to process 419031 (/home/sanjay42/sanjay/cuda/ATLAS_CUDA_tutorial/CUDA/02_Expose_Parallelism/vector_add)
Current device: 0
Device name: NVIDIA GeForce RTX 4060 Ti
Total global memory: 16453568 KB
Total constant memory: 64 KB
Total shared memory per block: 48 KB
==PROF== Profiling "vadd" - 0: 0%....50%....100% - 9 passes
A[0] = 0.840188
B[0] = 0.394383
C[0] = 1.234571
==PROF== Disconnected from process 419031
[419031] vector_add@127.0.0.1
  vadd(const float *, const float *, float *, int) (640, 1, 1)x(512, 1, 1), Context 1, Stream 7, Device 0, CC 8.9
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- ------------
    Metric Name               Metric Unit Metric Value
    ----------------------- ------------- ------------
    DRAM Frequency          cycle/nsecond         8.72
    SM Frequency            cycle/nsecond         2.30
    Elapsed Cycles                  cycle    3,522,912
    Memory Throughput                   %        91.

### e) Programmatically determine a good choice

What if you didn't already know how many SMs your GPU has? More importantly, what if you want to write code that is portable to all of the GPU architectures we've described? In that case, you want to be able to programmatically determine the number of blocks to launch (and possibly the number of threads per block) so that you can make a good choice. To obtain information about the device we're running on, we can use the CUDA API `cudaGetDeviceProperties()`:

```
// The API requires us to know which device we're interested in (there may be several on your system)
// cudaGetDevice() will tell us which GPU we're using. If there's just one GPU, it will be ID 0.
int device_id;
cudaGetDevice(&device_id);

// Now get the device properties and store them in a CUDA-defined struct, cudaDeviceProp.
cudaDeviceProp device_properties;
cudaGetDeviceProperties(&device_properties, device_id);
```

Modify the vector_add application so that the number of threadblocks is equal to the number of SMs multiplied by the maximum number of blocks per SM. (Use 32 threads per block for this case.) Does this yield reasonable performance? Also, as practice, insert code that checks to see your threadblock does not have more threads than is permitted by the device. The member of the struct that returns the number of SMs is called `multiProcessorCount`. Consult the [CUDA Runtime API documentation](https://docs.nvidia.com/cuda/cuda-runtime-api/structcudaDeviceProp.html) to figure out the names of the other quantities we need. A solution is shown in [solutions/vector_add.cu](solutions/vector_add.cu).

In [14]:
!nvcc -arch=native -o vector_add exercises/vector_add.cu; sudo ncu --section SpeedOfLight ./vector_add

==PROF== Connected to process 419956 (/home/sanjay42/sanjay/cuda/ATLAS_CUDA_tutorial/CUDA/02_Expose_Parallelism/vector_add)
Current device: 0
Device name: NVIDIA GeForce RTX 4060 Ti
Total global memory: 16453568 KB
Total constant memory: 64 KB
Total shared memory per block: 48 KB
==PROF== Profiling "vadd" - 0: 0%....50%....100% - 9 passes
A[0] = 0.840188
B[0] = 0.394383
C[0] = 1.234571
==PROF== Disconnected from process 419956
[419956] vector_add@127.0.0.1
  vadd(const float *, const float *, float *, int) (816, 1, 1)x(1024, 1, 1), Context 1, Stream 7, Device 0, CC 8.9
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- ------------
    Metric Name               Metric Unit Metric Value
    ----------------------- ------------- ------------
    DRAM Frequency          cycle/nsecond         8.77
    SM Frequency            cycle/nsecond         2.32
    Elapsed Cycles                  cycle    3,516,636
    Memory Throughput                   %        91

## Execution Model

With some understanding of the hardware microarchitecture in hand, we can turn to the CUDA execution model: how do parallel threads and blocks map to the hardware?

![](images/execution_model.png)

The mapping shown above should be understood loosely. For example, SP/DP units only handle certain mathematical operations and do not handle other instructions such as loads and stores. Still, we can always define a one-to-one correspondence between the instruction a thread is executing and some functional unit on the SM.

###  Warps

![](images/warps.png)

## Launch Configuration

So, what does all of this mean for optimal kernel launch configuration? Recall that we previously defined the launch configuration as the arguments between the angle brackets in a kernel launch:


```
kernel<<<N,M>>>();
```

where `N` is the number of blocks and `M` is the number of threads per block.

We've already suggested that the primary answer is that we want to launch many threads. (The total number of threads in a grid/kernel launch is simply `N * M`.) Why is it that the GPU needs to have a lot of threads from a performance perspective? Let's discuss about some relevant facts:

- Instructions are issued (warp-wide) in order


- A thread stalls when one of the operands (input data) isn’t ready

  - Note that a memory read by itself doesn’t generally stall execution (we can issue a read instruction, and then go on and do other work)


- Latency is hidden by switching threads

  - Global memory latency: >100 cycles  (varies by architecture/design)

  - Arithmetic latency: <100 cycles  (varies by architecture/design)


So, how many threads/blocks should we launch? The short answer is: **we need to launch enough threads to hide latency**.

## GPU Latency Hiding

Let's examine a very brief snippet of CUDA C source code:

```
int idx = threadIdx.x + blockDim.x * blockIdx.x;

c[idx] = a[idx] * b[idx];
```

What might this look like in machine code?

```
I0: LD R0, a[idx]
I1: LD R1, b[idx]
I2: MPY R2, R0, R1
```

**Use this slider below to step through an example of GPU latency hiding**. Run the cell to make the slider appear, then drag left to right to expose the images. The content starts from 1 and end on 10. After reading through slide 10, please continue on to the next paragraphs.

In [15]:
import ipywidgets as widgets
from IPython.display import Image, display, clear_output

listOfLatency = ['images/latency_hiding_1.png',
                'images/latency_hiding_2.png',
                'images/latency_hiding_3.png',
                'images/latency_hiding_4.png',
                'images/latency_hiding_5.png',
                'images/latency_hiding_6.png',
                'images/latency_hiding_7.png',
                'images/latency_hiding_8.png',
                'images/latency_hiding_9.png',
                'images/latency_hiding_10.png']
        
w2 = widgets.IntSlider(0,0,10,1)

#display(Image(filename=listOfLatency[w2.value - 1]))  
    
def on_value_change(change):
    with out:  
        clear_output()
        display(Image(filename=listOfLatency[change['new'] -1])) 
        if ((change['new'] -1) == 0):
            print ("Now how does this work in hardware? Let's draw a two dimensional chart where the horizontal axis is clock cycles (starting from clock cycle 0, C0) and the vertical axis is warps (W0 is the first warp, W1 is the second warp, etc.). In this exercise we're going to pretend we're the warp scheduler and determine what instructions to issue to what warps on which clock cycles.")
        if ((change['new'] -1) == 1):
            print ("The GPU SM has an in-order execution model, so we have to start by issuing instruction I0 to a warp on cycle C0. We can select any of our 10 warps in this toy model; it doesn't matter which one we choose, so let's assume we issue this to warp W0.")
        if ((change['new'] -1) == 2):
            print ("Let's assume that this fully occupies the machine (no other work can be performed on C0), so we proceed to cycle C1. We now have a similar choice to make. We could issue instruction I1 to W0, or we could issue I0 to a different warp. Those are both valid choices and the machine could do either. For the purposes of pedagogy let's assume we issue instruction I1 to the same warp (W0) on this clock cycle. We know this is fair because, as we stated earlier, load instructions never stall, and I0 and I1 are both load instructions.\
The next instruction is I2. Can we issue that immediately in cycle C2 to warp W0? No. Instruction I2 depends on having registers R0 and R1 already populated, but this has not happened yet, because the latency of memory reads is longer than one cycle. So warp W0 is stalled at this point.")
        if ((change['new'] -1) == 3):
            print ("At this point, we no longer have an option but to switch to another warp. Let's issue I0 to W1.")
        if ((change['new'] -1) == 4):
            print ("Since this warp is not stalled, we can immediately issue I1.")
        if ((change['new'] -1) == 5):
            print ("Warp W1 is now stalled, so we can switch to the next warp, which will then stall after issuing I0 and I1.")
        if ((change['new'] -1) == 6):
            print ("We repeat this process across many warps; we could continue this process as long as there are warps available.")
        if ((change['new'] -1) == 7):
            print ("Our ultimate goal here is to keep the machine busy, which we can define as ensuring that there is no clock cycle in which the processor is idling (an instruction is issued every cycle). However, eventually, one of two things will happen. Either we'll run out of warps, or the latency of a previous operation will expire. At some point, the memory subsystem will deliver the value of a[idx] into register R0 as requested by instruction I0. A similar statement is true for instruction 11: at some point, we will have loaded b[idx] into registers.")
        if ((change['new'] -1) == 8):
            print ("When warp W0 is no longer stalled because the latency of its previous operations has expired (that is, the operands for instruction I2 are loaded into registers), a new operation is possible: we can issue instruction I2 into warp W0.")
        if ((change['new'] -1) == 9):
            print ("Generally speaking, our memory operations are pipelined and have a fixed latency. A consequence of that is that if on some clock cycle we load the result from instruction I0 into register R0 for warp W0, we know that on the next clock cycle, we'll receive the data for instruction I1 into register R1 for that same warp. In the next clock cycle, we'll receive the data for R0 in warp W1, and then likewise for R1 in warp W1. So at this point warp W1 will no longer be stalled, and we can issue instruction I2 in warp W1.")


w2.observe(on_value_change, names='value')    
    
out=widgets.Output()
display(w2)
display(out)

IntSlider(value=0, max=10)

Output()

We will then continue on that way for all warps.

So the question to think about here is: given this architecture, how can we as the programmer optimize this scenario? That is, how can we help the warp scheduler do the most efficient possible job? The simple answer is: make the vertical axis longer. That is, if instead of having 10 warps we had 20 warps, we could go twice as long in this sequence before stalling, and if we had 40 warps we could go even farther. Since latencies can be much longer than 10 or 20 clock cycles, we may need a much longer vertical axis to minimize the risk of stalling. In the asymptotic limit of infinite work to do, and infinite warps, we could always keep the machine busy, though that's not a realistic case. In reality we have a hardware limit on the number of warps that can be resident on an SM. For all recent GPUs targeted at scientific high performance computing (e.g. K20X, P100, V100, and A100), this limit is 64 warps per SM.

So our ultimate takeaway is: **we need to have a lot of warps per SM**. Ideally we can launch this upper limit of warps. Since there are 32 threads per warp, this translates into 2048 threads per SM. For the V100 GPU with 80 SMs, a fully occupied GPU would have 80 * 2048 = 163,840 threads active. So if you write a CUDA kernel that fields fewer than that number of threads, we can say that you are not saturating the GPU, and so you are very likely not obtaining its full compute capability. When we launch more threads, we have a better ability to hide latency, and we are more likely to succeed.

## Launch Configuration

Returning to our concept of optimal launch configuration: suppose we want to hide arithmetic latency, which occurs on the scale of O(10) cycles. Then we probably need only O(10) warps (~320 threads) to have a good chance of hiding arithmetic latency. However, global memory latency is much larger than arithmetic latency, so in reality we want as many warps as we can to hide latency.

We also will want to maximize global memory throughput. It turns out that there's a relationship between hiding latency by launching many threads, and achievable memory throughput, as we'll see in the exercise below.

There are some things we can do as a programmer to improve memory throughput from device memory. Throughput in general depends on access patterns and word sizes, and if we can access larger chunks of data at one time, and/or ensure those chunks are coalesced, we'll get higher throughput.

### Maximizing Memory Throughput

Here's one illustration of this concept that, when a thread can issue fewer memory load instructions because the word size is larger, we obtain higher throughput by saturating the memory bus with a fewer number of threads per SM.

![](images/maximizing_memory_throughput.png)

## Exercise: A Compute-Bound Problem

Let's look at an artificially compute-bound example. The code [exercises/compute.cu](exercises/compute.cu) fills an array of values with the extended product of the integers from 1 to 100. The product is computed explicitly as an unrolled loop of 100 sequential floating point multiplies. Run this example with several different choices for grid and block size. How many threads are needed to achieve a reasonable fraction of the peak floating point compute throughput (`SM %`, as reported by Nsight Compute) of the GPU?

In [24]:
#with float precision, unrolled 100 times
!nvcc -arch=native -o compute exercises/compute.cu; sudo ncu --section SpeedOfLight ./compute

==PROF== Connected to process 427145 (/home/sanjay42/sanjay/cuda/ATLAS_CUDA_tutorial/CUDA/02_Expose_Parallelism/compute)
==PROF== Profiling "kernel(float *, int)" - 0: 0%....50%....100% - 9 passes
==PROF== Disconnected from process 427145
[427145] compute@127.0.0.1
  kernel(float *, int) (4096, 1, 1)x(1024, 1, 1), Context 1, Stream 7, Device 0, CC 8.9
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- ------------
    Metric Name               Metric Unit Metric Value
    ----------------------- ------------- ------------
    DRAM Frequency          cycle/nsecond         8.71
    SM Frequency            cycle/nsecond         2.30
    Elapsed Cycles                  cycle    2,219,081
    Memory Throughput                   %        93.59
    DRAM Throughput                     %        93.59
    Duration                      usecond       964.45
    L1/TEX Cache Throughput             %        13.25
    L2 Cache Throughput                 %        17.1

Now using your choice for the grid configuration from above, vary the amount of compute work in the inner for loop in the kernel (decrease or increase the number of floating point multiplies). How does that affect the performance, for a fixed number of threads? 

Estimate the arithmetic intensity of the kernel for a given configuration -- the ratio of floating point operations to bytes loaded/stored -- and determine the point at which the arithmetic intensity for this problem becomes low enough to transition from compute-bound to memory bandwidth-bound.

Does the arithmetic intensity change if we use double precision data and double precision multiplies? Try switching from float to double and see what happens.

In [27]:
#with float precision, unrolled 1000 times
!nvcc -arch=native -o compute exercises/compute.cu; sudo ncu --section SpeedOfLight ./compute

==PROF== Connected to process 427958 (/home/sanjay42/sanjay/cuda/ATLAS_CUDA_tutorial/CUDA/02_Expose_Parallelism/compute)
==PROF== Profiling "kernel(float *, int)" - 0: 0%....50%....100% - 9 passes
==PROF== Disconnected from process 427958
[427958] compute@127.0.0.1
  kernel(float *, int) (4096, 1, 1)x(1024, 1, 1), Context 1, Stream 7, Device 0, CC 8.9
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- ------------
    Metric Name               Metric Unit Metric Value
    ----------------------- ------------- ------------
    DRAM Frequency          cycle/nsecond         8.74
    SM Frequency            cycle/nsecond         2.31
    Elapsed Cycles                  cycle    8,090,013
    Memory Throughput                   %        25.70
    DRAM Throughput                     %        25.70
    Duration                      msecond         3.50
    L1/TEX Cache Throughput             %         3.71
    L2 Cache Throughput                 %         4.8

In [ ]:
#with double precision, unrolled 1000 times
!nvcc -arch=native -o compute exercises/compute.cu; sudo ncu --section SpeedOfLight ./compute

==PROF== Connected to process 427692 (/home/sanjay42/sanjay/cuda/ATLAS_CUDA_tutorial/CUDA/02_Expose_Parallelism/compute)
==PROF== Profiling "kernel(double *, int)" - 0: 0%....50%....100% - 9 passes
==PROF== Disconnected from process 427692
[427692] compute@127.0.0.1
  kernel(double *, int) (4096, 1, 1)x(1024, 1, 1), Context 1, Stream 7, Device 0, CC 8.9
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- --------------
    Metric Name               Metric Unit   Metric Value
    ----------------------- ------------- --------------
    DRAM Frequency          cycle/nsecond           8.74
    SM Frequency            cycle/nsecond           2.31
    Elapsed Cycles                  cycle    588,699,774
    Memory Throughput                   %           1.03
    DRAM Throughput                     %           1.03
    Duration                      msecond         254.85
    L1/TEX Cache Throughput             %           0.09
    L2 Cache Throughput        

### Summary

We need **enough total threads to keep the GPU busy**. Typically we'd like 512+ threads per SM (but aim for 2048, which achieves maximum "occupancy"). Some exceptions to this general rule of thumb do exist, due to various limiters other than latency on the GPU, and so in some cases increasing the number of threads per SM will not increase your performance. When we dig into analysis-driven optimization later, we'll see that.

Let's consider what all this means for threadblock configuration. First, threads per block should be a **multiple of the warp size (32)**. There's no benefit to other choices, because instructions are always issued in groups exactly equal to the warp size. If we had 48 threads per block, we'd issue instructions to two warps with 32 threads each, but one of the warps would have 16 of its threads idling. This means we'd be wasting some of the processing power of the SM, compared to an alternate configuration where every warp has all of its threads active.

The recent high-end HPC-focused GPUs we are focused on (e.g. V100 or A100) can concurrently execute **32 blocks per SM**. In general, we want to use multiple blocks per SM, and in fact we must do so to achieve maximum performance (for example, a block is limited to 1024 threads, but we saw above that an SM can host up to 2048 threads). When thinking about the number of threads per block, there are tradeoffs to consider that we won't go into here, but may become relevant as you get further into CUDA optimization. Usually the optimal number of threads is empirically determined for each kernel. A typical place to start is **128 or 256 threads per block**, but you should always use whatever is best for your application.

### Aside: What is Occupancy?

We've used this term a couple times above. Occupancy is a measure of the actual thread load on the SM, versus the peak theoretically achievable load. For example if I can run 64 warps per SM and I'm only running 16, my occupancy is 25%. There are several limiters to theoretical occupancy (including number of registers used per thread, number of threads per threadblock, and amount of shared memory used per block), so it can be a little tricky to calculate achievable occupancy. CUDA provides an [occupancy calculator](https://docs.nvidia.com/cuda/cuda-occupancy-calculator/index.html) which is a spreadsheet you can use to help figure this out. The NVIDIA profiling tools also show this information to you.

## Review

In this lab, we learned that:

- GPUs are a massively thread-parallel, latency hiding machines


- We're focused on selecting the optimal kernel launch configuration to maximize performance
  - Launch enough threads per SM to hide latency
  - Launch enough threadblocks to load the GPU


Also, note that:

- We should use analysis/profiling when optimizing
  - Nsight Compute can show you information about whether you’ve saturated the compute subsystem or the memory subsystem

## Further Study

[Optimization in-depth](http://on-demand.gputechconf.com/gtc/2013/presentations/S3466-Programming-Guidelines-GPU-Architecture.pdf)


[Analysis-Driven Optimization](http://on-demand.gputechconf.com/gtc/2012/presentations/S0514-GTC2012-GPU-Performance-Analysis.pdf)


[CUDA Best Practices Guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html)


[CUDA Tuning Guides](https://docs.nvidia.com/cuda/index.html#programming-guides) (Kepler/Maxwell/Pascal/Volta)

## Lab Materials

You can download this notebook using the `File > Download as > Notebook (.ipnyb)` menu item. Source code files can be downloaded from the `File > Download` menu item after opening them.